In [ ]:
%pip install httpx pandas pandas_ta matplotlib seaborn plotly requests beautifulsoup4 lxml mplfinance finnhub-python binance python-binance scikit-learn numpy rich marketwatch

In [ ]:
import time
import pandas as pd
from datetime import datetime
from marketwatch import MarketWatch
import logging
import os
import re

In [ ]:
def calculate_ema(price_series, periods):
    return price_series.ewm(span=periods, adjust=False).mean()

In [ ]:
def trade_on_crossover(df, game_name, stock_symbol, marketwatch):
    short_term = df['EMA_20'].iloc[-1]
    long_term = df['EMA_80'].iloc[-1]
    prev_short_term = df['EMA_20'].iloc[-2]
    prev_long_term = df['EMA_80'].iloc[-2]

    # Condition to buy or initiate a long position
    if short_term > long_term and prev_short_term <= prev_long_term:
        try:
            marketwatch.buy(game_name, stock_symbol, 100)
            logging.info(f"Bought 100 shares of {stock_symbol}")
        except Exception as e:
            logging.error(f"Error buying {stock_symbol}: {e}")

    # Condition to sell or close a long position
    elif short_term < long_term and prev_short_term >= prev_long_term:
        try:
            marketwatch.sell(game_name, stock_symbol, 100)
            logging.info(f"Sold 100 shares of {stock_symbol}")
        except Exception as e:
            logging.error(f"Error selling {stock_symbol}: {e}")

    # Additional conditions for short selling

    # Condition to initiate a short sell
    elif short_term < long_term and prev_short_term >= prev_long_term:
        try:
            marketwatch.short_sell(game_name, stock_symbol, 100)
            logging.info(f"Initiated a short sell of 100 shares of {stock_symbol}")
        except Exception as e:
            logging.error(f"Error initiating short sell of {stock_symbol}: {e}")

    # Condition to close a short position
    elif short_term > long_term and prev_short_term <= prev_long_term:
        try:
            marketwatch.cover_short(game_name, stock_symbol, 100)
            logging.info(f"Closed short position of 100 shares of {stock_symbol}")
        except Exception as e:
            logging.error(f"Error closing short position of {stock_symbol}: {e}")

In [ ]:
def trade_multiple_stocks(stock_symbols, marketwatch, game_name):
    price_data = {symbol: [] for symbol in stock_symbols}

    for _ in range(160):
        for symbol in stock_symbols:
            try:
                price_pattern = re.compile(r'\b' + re.escape(symbol) + r' : \$([\d\.]+)\b')
                price_string = marketwatch.get_price(symbol)
                price_match = price_pattern.search(price_string)
                if price_match:
                    price = float(price_match.group(1))
                    price_data[symbol].append(price)
                else:
                    print(f"Received price string: {price_string}")
                    raise ValueError(f"Unexpected price format for {symbol}: {price_string}")
            except Exception as e:
                print(f"Unable to parse price for symbol: {symbol}, received: {price_string}, error: {e}")
                continue

            if len(price_data[symbol]) > 150:
                price_data[symbol].pop(0)

        for symbol in stock_symbols:
            df = pd.DataFrame(price_data[symbol], columns=['Close'])
            for period in [20, 80]:
                ema_column_name = f'EMA_{period}'
                df[ema_column_name] = float('nan')
            for period in [20, 80]:
                ema_column_name = f'EMA_{period}'
                df[ema_column_name] = calculate_ema(df['Close'], period)

            if len(df) >= 80:
                trade_on_crossover(df, game_name, symbol, marketwatch)
        time.sleep(1)  # Adjust this based on MarketWatch data update frequency
        print(df)

In [ ]:
def next_trading_day():
    today = datetime.today()
    if today.weekday() >= 5:  # Saturday or Sunday
        return today + timedelta(days=(7 - today.weekday()))
    return today

In [ ]:
def main():
    username = os.getenv('MARKETWATCH_USERNAME')
    password = os.getenv('MARKETWATCH_PASSWORD')
    if not username or not password:
        logging.error("Username or password not set in environment variables.")
        return

    game_name = 'marketwatchapistrategieema'
    marketwatch = MarketWatch(username, password)

    next_trade_day = next_trading_day()
    if datetime.today() < next_trade_day:
        sleep_seconds = (next_trade_day - datetime.today()).total_seconds()
        logging.info(f"Today is a non-trading day. Sleeping until {next_trade_day}")
        time.sleep(sleep_seconds)

    tech_stocks = ['AAPL', 'GOOGL']
    health_stocks = ['JNJ']
    utility_stocks = ['NEE']
    all_stocks = tech_stocks + health_stocks + utility_stocks

    trade_multiple_stocks(all_stocks, marketwatch, game_name)

if __name__ == "__main__":
    main()